Ingestion - Input data and chunk them

In [1]:
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader, PyPDFLoader
import pickle  

# Path to your data folder
data_dir = "pre_chosen_sources"  
# List to hold all chunks
all_chunks = []
source_map = {}

#Getting all files
files = [f for f in os.listdir(data_dir) if f.lower().endswith(('.txt', '.pdf'))] # only support text file and pdf file for data extraction

for id, file_name in enumerate(files, start=1):
    file_path = os.path.join(data_dir, file_name)
    source_map[id] = file_name

    #Load according to the file extension
    if file_name.lower().endswith(".txt"):
        loader = TextLoader(file_path, encoding="utf-8")
    elif file_name.lower().endswith(".pdf"):
        loader = PyPDFLoader(file_path)
    else:
        continue # skip loop if file is not supported

    docs = loader.load()
    text = " ".join([doc.page_content for doc in docs])
    text = ' '.join(text.split())  # Collapses multiples
    text = text.replace('\n', ' ')  # Flatten newlines if any

    splitter = RecursiveCharacterTextSplitter(
            chunk_size=500,  # ~500 chars/tokens
            chunk_overlap=50  # Overlap for context continuity

        )
    file_chunks = splitter.split_text(text)
    for chunk in file_chunks:
        all_chunks.append({'text': chunk, 'source_id': id}) #updated to cite according to sources' name
    
    print(f"Processed {file_name[:10]} ... as source_id {id}, added {len(file_chunks)} chunks")


# Final count
print(f"Total chunks created: {len(all_chunks)}")

# Save data 
with open("chunks.pkl", "wb") as f:
    pickle.dump(all_chunks, f)
with open("source_map.pkl", "wb") as f:
    pickle.dump(source_map, f)

print("Data retrieval is complete")

c:\Users\nguye\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Processed article_ag ... as source_id 1, added 21 chunks
Processed article_sa ... as source_id 2, added 7 chunks
Processed article_th ... as source_id 3, added 88 chunks
Processed diggit_mag ... as source_id 4, added 32 chunks
Processed editorial_ ... as source_id 5, added 19 chunks
Processed fullbright ... as source_id 6, added 6 chunks
Processed In_search_ ... as source_id 7, added 46 chunks
Processed reuters_th ... as source_id 8, added 7 chunks
Processed saigoneer. ... as source_id 9, added 17 chunks
Processed song_your_ ... as source_id 10, added 3 chunks
Processed TCS-Buddhi ... as source_id 11, added 198 chunks
Processed thenewlens ... as source_id 12, added 10 chunks
Processed the_new_yo ... as source_id 13, added 11 chunks
Processed Trinh Cong ... as source_id 14, added 19 chunks
Processed Trinh_Cong ... as source_id 15, added 9 chunks
Processed trinh_cong ... as source_id 16, added 3 chunks
Processed trinh_cong ... as source_id 17, added 3 chunks
Processed Vietcetera ... as s

In [7]:
# from transformers import pipeline, set_seed

# generator = pipeline('question-answering', model='gpt2')
# set_seed(2025)

from huggingface_hub import InferenceClient


client = InferenceClient(model="meta-llama/Meta-Llama-3.1-405B-Instruct")


Load processed text

In [2]:
import pickle
with open("chunks.pkl", "rb") as f:
    all_chunks = pickle.load(f)
print(f"Loaded {len(all_chunks)} chunks")  # e.g., 30 chunks

Loaded 574 chunks


Embed the chunks above

In [3]:
from sentence_transformers import SentenceTransformer
import numpy as np
model = SentenceTransformer('all-MiniLM-L6-v2')


embed_model = SentenceTransformer('all-MiniLM-L6-v2') 
embeddings = embed_model.encode(all_chunks)  #
embeddings = np.array(embeddings).astype('float32')  # FAISS needs this
print(f"Embeddings shape: {embeddings.shape}")  # e.g., (30, 384) – chunks x dimensions

Embeddings shape: (574, 384)


Using Faiss for fast vector database searching

In [4]:
import faiss

dimension = embeddings.shape[1]  # e.g., 384
index = faiss.IndexFlatL2(dimension)  # Basic flat index (exact search, good for small data)
index.add(embeddings)  # Train/add your vectors
print(f"Index built with {index.ntotal} vectors")  # Matches chunk count

Index built with 574 vectors


Chat completion and comparision between two pipelines

In [8]:
import pandas as pd
import time
from pipelines import * 
import re
from sentence_transformers import util
import numpy as np
import pickle  


def augment_prompt_no_query(retrieved_chunks):
    context_parts = []
    for chunk_tuple in retrieved_chunks:
        chunk_dict = chunk_tuple[0]  # {'text':, 'source_id':}
        source_id = chunk_dict['source_id']
        text = chunk_dict['text']
        context_parts.append(f"[Source ID: {source_id}]\n{text}")
    
    context = "\n\n".join(context_parts)
    prompt = f"""Use the following context to answer the question factually. If the context doesn't cover it, say "I don't have info on that. You MUST always provide a response—do not return nothing or an empty answer"
    
            Context: {context}"""
    return prompt  # this is prompt engineering

 # Use embedding model for Post-Generation Similarity Matching in conjunction with traditional regex searching
def attribute_sources(ragged_output, retrieved_chunks, embed_model, threshold=0.7):
    if not ragged_output:
        return []
    

    sentences = [s.strip() for s in ragged_output.split('.') if s.strip()]
    
    if not sentences:
        return []
    
    # Embed response sentences
    sent_embeds = embed_model.encode(sentences)
    
    # Embed retrieved chunks
    chunk_texts = [chunk_tuple[0]['text'] for chunk_tuple in retrieved_chunks]
    chunk_embeds = embed_model.encode(chunk_texts)
    
    used_source_ids = set()
    for sent_embed in sent_embeds:
        similarities = util.cos_sim(sent_embed, chunk_embeds)[0]
        max_sim_idx = np.argmax(similarities)
        if similarities[max_sim_idx] >= threshold:
            source_id = retrieved_chunks[max_sim_idx][0]['source_id']
            used_source_ids.add(source_id)
    
    return sorted(used_source_ids)

def compare_ragged_outputs(queries, embed_model, index, all_chunks, max_length=300):
    with open("source_map.pkl", "rb") as f:
        SOURCE_MAP = pickle.load(f)
    results = []
    for query in queries:
        start_time = time.time()
        
        # Plain LLM
        plain_messages = [{"role": "system", "content": "You MUST always provide a response—do not return nothing or an empty answer"}, {"role": "user", "content": query}]
        plain_response = client.chat_completion(plain_messages, max_tokens=300)
        plain_output = plain_response.choices[0].message.content  # Extract content
        plain_time = time.time() - start_time

        if plain_output is None: # can still return None if there is error in API call
            plain_output = ""  # Treat as empty string
            print(f"Warning: LLM returned None for query '{query}'—handling as empty.")

        # RAG
        rag_start = time.time()
        retrieved = retrieve_chunks(query, embed_model, index, all_chunks)
        rag_messages = [{"role": "system", "content": augment_prompt_no_query(retrieved)}, {"role": "user", "content": query}]  # System first
        rag_response = client.chat_completion(rag_messages, max_tokens=300)
        ragged_output = rag_response.choices[0].message.content
        used_sources = set()
        
        if ragged_output is None:
            ragged_output = ""  # Treat as empty string
            print(f"Warning: RAG model returned None for query '{query}'—handling as empty.")

        # Step 1: Try regex parsing for inline citations (as fallback or hybrid)
        used_source_ids_regex = set(re.findall(r'\[Source ID: \d+\]', ragged_output))  # Fixed regex to match [Source ID: x]
        
        # Step 2: Use similarity matching as primary method
        used_source_ids_sim = set(attribute_sources(ragged_output, retrieved, embed_model))
        
        # Combine: Use similarity primarily, add regex if any unique
        used_source_ids = used_source_ids_sim.union(used_source_ids_regex)
        
        # Map to file names
        used_sources = {SOURCE_MAP[int(sid)] for sid in used_source_ids if int(sid) in SOURCE_MAP}
        
        # If no citations found, fall back to all retrieved sources
        if not used_sources:
            used_source_ids_all = set(chunk_tuple[0]['source_id'] for chunk_tuple in retrieved)
            used_sources = {SOURCE_MAP[sid] for sid in used_source_ids_all if sid in SOURCE_MAP}
        
        # Sort and format
        sources_list = sorted(used_sources)
        sources_str = "Sources: " + ", ".join(sources_list) if sources_list else "No sources used."
        
        # Append to ragged_output
        full_output = ragged_output + "\n\n" + sources_str if ragged_output else "No output generated."

        rag_time = time.time() - rag_start
        
        results.append({
            'query': query,
            'plain_answer': plain_output,
            'rag_answer': full_output,
            'plain_latency': plain_time,
            'rag_latency': rag_time
        })
        print(f"Processed: {query} | Plain: {plain_time:.2f}s | RAG: {rag_time:.2f}s")
    
    # Save to CSV if you want
    df = pd.DataFrame(results)
    df.to_csv('comparison_results.csv', index=False)
    print("Results saved to comparison_results.csv")
    return df

TEST SPACE


In [9]:
#TEST
# uncomment to test
test_queries = [
        "What was the first song made by Trinh Cong Son?",
        "Where was Trinh Cong Son born?", 
        "How many songs did Trinh Cong Son produce?",
        "Trinh Cong Son was nickname?", 
        "Trinh Cong Son favorite song is ?", 
        "How influential was Trinh Cong Son's legacy",
        "Was Trinh Cong Son consider 'Bob Dylan 'of Vietnam",
        "DId Trinh Cong Son write antiwar songs?"]
df = compare_ragged_outputs(test_queries, embed_model, index, all_chunks, max_length=100)
df
#RAG shows no hallucination and cite its sources. Plain LLM hallucinates more. 

Processed: What was the first song made by Trinh Cong Son? | Plain: 4.84s | RAG: 1.90s
Processed: Where was Trinh Cong Son born? | Plain: 1.27s | RAG: 1.45s
Processed: How many songs did Trinh Cong Son produce? | Plain: 2.35s | RAG: 1.97s
Processed: Trinh Cong Son was nickname? | Plain: 1.43s | RAG: 1.19s
Processed: Trinh Cong Son favorite song is ? | Plain: 2.58s | RAG: 1.48s
Processed: How influential was Trinh Cong Son's legacy | Plain: 4.96s | RAG: 4.44s
Processed: Was Trinh Cong Son consider 'Bob Dylan 'of Vietnam | Plain: 4.57s | RAG: 2.22s
Processed: DId Trinh Cong Son write antiwar songs? | Plain: 4.25s | RAG: 1.62s
Results saved to comparison_results.csv


,query,plain_answer,rag_answer,plain_latency,rag_latency
0,What was the first song made by Trinh Cong Son?,"Unfortunately, I couldn't find any reliable in...","According to the context, Trinh Cong Son's fir...",4.841830,1.899735
1,Where was Trinh Cong Son born?,"Trinh Cong Son was born in Dac Lac Province, V...",Trinh Cong Son was born in the former royal ci...,1.266190,1.446827
2,How many songs did Trinh Cong Son produce?,Trinh Cong Son is a renowned Vietnamese musici...,I don't have info on that. The provided contex...,2.350685,1.967164
3,Trinh Cong Son was nickname?,"Trinh Cong Son was often nicknamed the ""Bob Dy...",I don't have info on that.\n\nSources: article...,1.431026,1.189382
4,Trinh Cong Son favorite song is ?,"Unfortunately, I couldn't find any reliable so...",I don't have info on Trinh Cong Son's favorite...,2.582934,1.481032
5,How influential was Trinh Cong Son's legacy,Trinh Cong Son (1939-2001) was a renowned Viet...,"According to the context, Trinh Cong Son's leg...",4.959773,4.441008
6,Was Trinh Cong Son consider 'Bob Dylan 'of Vie...,"What a fascinating comparison!\n\nYes, Trinh C...",I don't have info on that. The provided contex...,4.569571,2.218799
7,DId Trinh Cong Son write antiwar songs?,"Yes, Trinh Cong Son was a Vietnamese musician ...","Yes, according to the context, Trinh Cong Son ...",4.250162,1.615284
